<a href="https://colab.research.google.com/github/jdanas/adk-essay-analyser/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and Setup

First, let's install the required dependencies for fine-tuning and the ADK Essay Analyzer project.

```bash
pip install -r requirements.txt
```

Next, download and install the ADK Essay Analyzer dependencies:

```bash
pip install adk-essay-analyzer
```

In [2]:
%%capture
import os

# Install Unsloth for efficient fine-tuning
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
    # Install Google ADK for agent management
    !pip install google-adk
else:
    # Colab-specific installations
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth
    # Install Google ADK for agent management
    !pip install google-adk

# Additional dependencies for the essay analyzer project
!pip install fastapi uvicorn python-multipart
!pip install pandas numpy matplotlib seaborn

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

## Loading the ADK Essay Analyzer Content Agent

Let's examine the Content Analyzer sub-agent that we'll be training. This agent specializes in analyzing essay content quality, argumentation, evidence usage, and critical thinking.

In [3]:
# Add the project path to sys.path so we can import our modules
import sys
import os
sys.path.append('../')  # Assuming notebook is in nb/ directory

# Import the content analyzer from our project
from essay_analyzer.sub_agents.content_analyzer import content_analyzer_agent

# Examine the content analyzer agent
print("Content Analyzer Agent Details:")
print(f"Name: {content_analyzer_agent.name}")
print(f"Model: {content_analyzer_agent.model}")
print(f"Description: {content_analyzer_agent.description}")
print(f"Output Key: {content_analyzer_agent.output_key}")
print("\nInstruction (first 300 chars):")
print(content_analyzer_agent.instruction[:300] + "...")

ModuleNotFoundError: No module named 'essay_analyzer'

## Creating Training Data for Content Analysis

Now let's create a dataset specifically for training the content analyzer. We'll use essay examples and expected analysis outputs that match the agent's expertise areas.

In [4]:
# Create sample essay analysis training data
import json

# Sample essays with expected content analysis outputs
content_analysis_data = [
    {
        "essay": """
Climate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.

The evidence for human-caused climate change is overwhelming. NASA data shows that global temperatures have risen by approximately 1.1 degrees Celsius since the late 19th century. Additionally, atmospheric CO2 levels have increased from 315 ppm in 1958 to over 420 ppm today. These changes correlate directly with increased industrial activity and fossil fuel consumption.

Critics often point to natural climate variations as an explanation for current changes. However, climate scientists have thoroughly examined these factors and concluded that natural forces alone cannot account for the rapid warming observed since the mid-20th century. The Intergovernmental Panel on Climate Change (IPCC) states with high confidence that human influence has been the dominant driver of observed warming.

To address this crisis, we must transition to renewable energy sources, improve energy efficiency, and implement carbon pricing mechanisms. Countries like Denmark and Costa Rica have demonstrated that economic growth and reduced emissions can occur simultaneously. Therefore, immediate action is both necessary and achievable.
        """,
        "analysis": """
**Content Analysis Report**

**Argument Quality and Logic (Score: 8/10)**
- Strong central thesis clearly identifying climate change as human-caused
- Logical progression from evidence presentation to addressing counterarguments
- Well-structured argument flow that builds convincingly
- Minor weakness: could benefit from more nuanced discussion of solution complexities

**Content Development (Score: 9/10)**
- Excellent topic relevance and focus
- Demonstrates solid understanding of climate science
- Good depth in exploring both evidence and counterarguments
- Specific examples enhance credibility

**Evidence and Source Usage (Score: 9/10)**
- Strong use of authoritative sources (NASA, IPCC)
- Specific data points effectively support claims
- Proper integration of evidence with analysis
- Could benefit from more diverse source types

**Critical Thinking (Score: 8/10)**
- Effectively addresses and refutes counterarguments
- Shows evaluation of different perspectives
- Moves beyond simple description to analysis
- Draws logical conclusions from evidence

**Audience Awareness (Score: 8/10)**
- Appropriate academic tone and formality
- Complex ideas communicated clearly
- Good balance of technical detail and accessibility

**Strengths:**
- Compelling use of specific data and statistics
- Effective counter-argument strategy
- Clear, logical structure
- Strong authoritative sources

**Areas for Improvement:**
- Could explore economic/social implications more deeply
- Might benefit from addressing implementation challenges
- Consider including more diverse perspectives on solutions
        """
    },
    {
        "essay": """
Social media is bad for society. Everyone uses it too much and it makes people sad and angry. My friend told me that she feels worse after looking at Instagram. Also, people spread fake news on these platforms.

I think we should limit social media use or maybe ban it completely. Some countries have tried to control it and it seems to work okay. People used to be happier before social media existed.

Studies show that social media causes depression and anxiety. It also makes people compare themselves to others too much. This is why young people have more mental health problems now than before.
        """,
        "analysis": """
**Content Analysis Report**

**Argument Quality and Logic (Score: 3/10)**
- Weak thesis statement lacks specificity and nuance
- Overly simplistic reasoning without acknowledgment of complexity
- Limited logical development between claims
- Sweeping generalizations without adequate support

**Content Development (Score: 2/10)**
- Superficial treatment of a complex topic
- Lacks depth and sophisticated analysis
- Minimal demonstration of understanding
- Relies heavily on personal anecdotes rather than broader evidence

**Evidence and Source Usage (Score: 2/10)**
- Vague reference to "studies" without specific citations
- Over-reliance on personal anecdote
- No credible sources provided
- Lacks integration of evidence with analysis

**Critical Thinking (Score: 2/10)**
- Limited analysis beyond surface-level observations
- No consideration of counterarguments or alternative perspectives
- Fails to examine nuances or complexities
- Conclusions not well-supported by evidence

**Audience Awareness (Score: 4/10)**
- Informal tone may not be appropriate for academic context
- Simple language accessible but lacks sophistication
- Ideas presented in basic manner

**Major Weaknesses:**
- Lacks specific, credible evidence
- No consideration of social media benefits or nuanced perspectives
- Oversimplified solutions to complex problems
- Inadequate development of ideas

**Recommendations for Improvement:**
- Conduct research using credible academic sources
- Develop more nuanced thesis that acknowledges complexity
- Include counterarguments and alternative perspectives
- Provide specific examples and evidence
- Strengthen logical connections between ideas
- Consider both benefits and drawbacks of social media
        """
    }
]

# Convert to Gemma-3 conversation format for training
def create_training_conversations(data):
    conversations = []
    for item in data:
        conversation = {
            "conversations": [
                {
                    "from": "user",
                    "value": f"Please analyze the content quality of this essay:\n\n{item['essay']}"
                },
                {
                    "from": "assistant",
                    "value": item['analysis']
                }
            ]
        }
        conversations.append(conversation)
    return conversations

training_conversations = create_training_conversations(content_analysis_data)

print(f"Created {len(training_conversations)} training conversations")
print("\nSample conversation structure:")
print(json.dumps(training_conversations[0], indent=2)[:500] + "...")

Created 2 training conversations

Sample conversation structure:
{
  "conversations": [
    {
      "from": "user",
      "value": "Please analyze the content quality of this essay:\n\n\nClimate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.\n\nThe evidence for human-caused climate change is overwhelming. NASA data shows that ...


In [5]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [6]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### Data Preparation for Content Analysis Training

Now we'll create training data specifically for the Content Analyzer. We need to format the data to match how the Content Analyzer works:

1. **Input**: Essay text
2. **Output**: Detailed content analysis following the agent's instruction format

We'll use the Gemma-3 conversation format where the user provides an essay and the model responds with detailed content analysis.

```
<bos><start_of_turn>user
Please analyze the content quality of this essay: [ESSAY TEXT]<end_of_turn>
<start_of_turn>model
[DETAILED CONTENT ANALYSIS]<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [7]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [8]:
# Create custom essay analysis training data
import json
from datasets import Dataset

# Sample essays with detailed content analysis that matches our Content Analyzer agent
essay_analysis_data = [
    {
        "essay": """Climate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.

The evidence for human-caused climate change is overwhelming. NASA data shows that global temperatures have risen by approximately 1.1 degrees Celsius since the late 19th century. Additionally, atmospheric CO2 levels have increased from 315 ppm in 1958 to over 420 ppm today. These changes correlate directly with increased industrial activity and fossil fuel consumption.

Critics often point to natural climate variations as an explanation for current changes. However, climate scientists have thoroughly examined these factors and concluded that natural forces alone cannot account for the rapid warming observed since the mid-20th century. The Intergovernmental Panel on Climate Change (IPCC) states with high confidence that human influence has been the dominant driver of observed warming.

To address this crisis, we must transition to renewable energy sources, improve energy efficiency, and implement carbon pricing mechanisms. Countries like Denmark and Costa Rica have demonstrated that economic growth and reduced emissions can occur simultaneously. Therefore, immediate action is both necessary and achievable.""",
        "analysis": """**Content Analysis:**

**Argument Quality and Logic (Score: 8/10)**
The essay presents a strong central thesis clearly identifying climate change as human-caused. The logical progression moves effectively from evidence presentation to addressing counterarguments and proposing solutions. The argument builds convincingly through specific data points to reach well-supported conclusions. Minor weakness: could benefit from more nuanced discussion of solution implementation complexities.

**Content Development (Score: 9/10)**
Excellent topic relevance and focus throughout. Demonstrates solid understanding of climate science fundamentals. Good depth in exploring both supporting evidence and potential counterarguments. The inclusion of specific examples (Denmark, Costa Rica) enhances credibility and shows practical application of proposed solutions.

**Evidence and Source Usage (Score: 9/10)**
Strong use of authoritative sources including NASA and IPCC data. Specific quantitative data points (1.1°C temperature rise, CO2 levels from 315 to 420+ ppm) effectively support central claims. Evidence is well-integrated with analysis rather than simply cited. Could benefit from additional diverse source types beyond scientific data.

**Critical Thinking (Score: 8/10)**
Effectively addresses and refutes counterarguments about natural climate variations. Shows evaluation of different perspectives on causation. Analysis moves beyond simple description to draw logical conclusions from evidence. Demonstrates synthesis of complex scientific information.

**Audience Awareness (Score: 8/10)**
Appropriate academic tone and formality level. Complex scientific concepts communicated clearly for educated audience. Good balance of technical detail and accessibility.

**Strengths:**
- Compelling use of specific quantitative data
- Effective counter-argument strategy demonstrates critical thinking
- Clear logical structure enhances persuasiveness
- Strong integration of authoritative sources

**Areas for Improvement:**
- Could explore economic and social implications more deeply
- Might benefit from addressing implementation challenges of proposed solutions
- Consider including more diverse perspectives on solution approaches"""
    },
    {
        "essay": """Social media is bad for society. Everyone uses it too much and it makes people sad and angry. My friend told me that she feels worse after looking at Instagram. Also, people spread fake news on these platforms.

I think we should limit social media use or maybe ban it completely. Some countries have tried to control it and it seems to work okay. People used to be happier before social media existed.

Studies show that social media causes depression and anxiety. It also makes people compare themselves to others too much. This is why young people have more mental health problems now than before.""",
        "analysis": """**Content Analysis:**

**Argument Quality and Logic (Score: 2/10)**
The thesis statement lacks specificity and nuance. Arguments rely on sweeping generalizations without adequate support. Limited logical development between claims, with conclusions not well-connected to premises. Fails to acknowledge the complexity of social media's societal impact.

**Content Development (Score: 2/10)**
Superficial treatment of a complex topic that requires deeper analysis. Minimal demonstration of understanding beyond surface-level observations. Limited exploration of various aspects of social media's impact. Relies heavily on personal anecdotes rather than broader evidence or research.

**Evidence and Source Usage (Score: 1/10)**
Vague reference to "studies" without specific citations or details. Over-reliance on single personal anecdote as primary evidence. No credible sources provided to support major claims. Lacks integration of evidence with analytical framework.

**Critical Thinking (Score: 2/10)**
Limited analysis beyond surface-level observations. No consideration of counterarguments or alternative perspectives. Fails to examine nuances, benefits, or complexities of social media use. Conclusions not adequately supported by evidence presented.

**Audience Awareness (Score: 3/10)**
Informal tone may not be appropriate for academic context. Language lacks sophistication expected for serious analysis. Ideas presented in overly simplistic manner.

**Major Weaknesses:**
- Lacks specific, credible evidence to support claims
- No consideration of social media benefits or positive uses
- Oversimplified solutions to complex societal issues
- Inadequate development of ideas and arguments
- Heavy reliance on personal experience rather than research

**Recommendations for Improvement:**
- Conduct thorough research using credible academic sources
- Develop more nuanced thesis acknowledging complexity
- Include analysis of both benefits and drawbacks
- Provide specific examples and statistical evidence
- Address counterarguments and alternative perspectives
- Strengthen logical connections between ideas"""
    }
]

# Convert to conversation format for training
def create_content_analysis_conversations(data):
    conversations = []
    for item in data:
        conversation = {
            "conversations": [
                {
                    "from": "user",
                    "value": f"Please analyze the content quality of this essay:\n\n{item['essay']}"
                },
                {
                    "from": "assistant",
                    "value": item['analysis']
                }
            ]
        }
        conversations.append(conversation)
    return conversations

training_conversations = create_content_analysis_conversations(essay_analysis_data)

# Use our custom content analysis training data instead of generic dataset
from datasets import Dataset

# Convert our training conversations to a dataset
dataset = Dataset.from_list(training_conversations)

print(f"Dataset created with {len(dataset)} examples")
print(f"Dataset features: {dataset.features}")
print(f"\nFirst example:")
print(dataset[0])

Dataset created with 2 examples
Dataset features: {'conversations': [{'from': Value(dtype='string', id=None), 'value': Value(dtype='string', id=None)}]}

First example:
{'conversations': [{'from': 'user', 'value': 'Please analyze the content quality of this essay:\n\nClimate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.\n\nThe evidence for human-caused climate change is overwhelming. NASA data shows that global temperatures have risen by approximately 1.1 degrees Celsius since the late 19th century. Additionally, atmospheric CO2 levels have increased from 315 ppm in 1958 to over 420 ppm today. These changes correlate directly with increased industrial activity and fossil fuel consumption.\n\nCritics often point to natural climate variations as an expla

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

## Testing the Original Content Analyzer

Before training, let's test the original Gemini-based content analyzer on our sample essay to see its baseline performance. This will help us compare with the fine-tuned model later.

In [9]:
# Test the original Content Analyzer on a sample essay
sample_essay = """
Technology has fundamentally transformed education in the 21st century. From online learning platforms to interactive whiteboards, digital tools have become integral to modern classrooms. While these changes offer numerous benefits, they also present significant challenges that educators must address.

The advantages of educational technology are substantial. Students can access vast amounts of information instantly, engage with interactive content, and learn at their own pace through personalized platforms. Remote learning capabilities, highlighted during the COVID-19 pandemic, have made education more accessible to students regardless of geographical constraints.

However, the digital divide remains a critical concern. Not all students have equal access to technology or reliable internet connections, potentially exacerbating educational inequalities. Additionally, excessive screen time may impact students' attention spans and social development.

Effective integration of technology in education requires thoughtful planning and adequate teacher training. Rather than replacing traditional teaching methods entirely, technology should complement and enhance human instruction to create more engaging and effective learning experiences.
"""

print("Sample essay for analysis:")
print("=" * 50)
print(sample_essay)
print("=" * 50)

# Note: In a real scenario, you would run the agent like this:
# result = content_analyzer_agent.run(text=sample_essay)
# print("Content Analysis Result:")
# print(result['content_analysis'])

print("\nThis would normally call the Gemini-based Content Analyzer.")
print("After training, we'll compare the fine-tuned Gemma-3 model's performance!")

Sample essay for analysis:

Technology has fundamentally transformed education in the 21st century. From online learning platforms to interactive whiteboards, digital tools have become integral to modern classrooms. While these changes offer numerous benefits, they also present significant challenges that educators must address.

The advantages of educational technology are substantial. Students can access vast amounts of information instantly, engage with interactive content, and learn at their own pace through personalized platforms. Remote learning capabilities, highlighted during the COVID-19 pandemic, have made education more accessible to students regardless of geographical constraints.

However, the digital divide remains a critical concern. Not all students have equal access to technology or reliable internet connections, potentially exacerbating educational inequalities. Additionally, excessive screen time may impact students' attention spans and social development.

Effective

In [12]:
from unsloth.chat_templates import standardize_data_formats

# Standardize our custom essay analysis data
dataset = standardize_data_formats(dataset)

print("Dataset structure after standardization:")
print(f"Features: {dataset.features}")
print(f"Number of examples: {len(dataset)}")
print("\nFirst example after standardization:")
print(dataset[0])

# Test the original content analyzer on our sample essay
sample_essay = """
Technology has fundamentally transformed education in the 21st century. From online learning platforms to interactive whiteboards, digital tools have become integral to modern classrooms. While these changes offer numerous benefits, they also present significant challenges that educators must address.

The advantages of educational technology are substantial. Students can access vast amounts of information instantly, engage with interactive content, and learn at their own pace through personalized platforms. Remote learning capabilities, highlighted during the COVID-19 pandemic, have made education more accessible to students regardless of geographical constraints.

However, the digital divide remains a critical concern. Not all students have equal access to technology or reliable internet connections, potentially exacerbating educational inequalities. Additionally, excessive screen time may impact students' attention spans and social development.

Effective integration of technology in education requires thoughtful planning and adequate teacher training. Rather than replacing traditional teaching methods entirely, technology should complement and enhance human instruction to create more engaging and effective learning experiences.
"""

print("Sample essay for content analysis:")
print("=" * 60)
print(sample_essay)
print("=" * 60)

# Note: In a real implementation, you would call the agent like this:
# result = content_analyzer_agent.run(text=sample_essay)
# print("\\nOriginal Content Analyzer Result:")
# print(result['content_analysis'])

print("\nThis represents the baseline performance of the Gemini-based Content Analyzer.")
print("After fine-tuning, we'll compare the Gemma-3-4B model's performance against this baseline!")

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset structure after standardization:
Features: {'conversations': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}]}
Number of examples: 2

First example after standardization:
{'conversations': [{'content': 'Please analyze the content quality of this essay:\n\nClimate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.\n\nThe evidence for human-caused climate change is overwhelming. NASA data shows that global temperatures have risen by approximately 1.1 degrees Celsius since the late 19th century. Additionally, atmospheric CO2 levels have increased from 315 ppm in 1958 to over 420 ppm today. These changes correlate directly with increased industrial activity and fossil fuel consumption.\n\nCritics often point to natur

Let's see how row 100 looks like!

In [14]:
dataset[0]

{'conversations': [{'content': 'Please analyze the content quality of this essay:\n\nClimate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.\n\nThe evidence for human-caused climate change is overwhelming. NASA data shows that global temperatures have risen by approximately 1.1 degrees Celsius since the late 19th century. Additionally, atmospheric CO2 levels have increased from 315 ppm in 1958 to over 420 ppm today. These changes correlate directly with increased industrial activity and fossil fuel consumption.\n\nCritics often point to natural climate variations as an explanation for current changes. However, climate scientists have thoroughly examined these factors and concluded that natural forces alone cannot account for the rapid warming observed si

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [15]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [16]:
dataset[0]["text"]

'<start_of_turn>user\nPlease analyze the content quality of this essay:\n\nClimate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.\n\nThe evidence for human-caused climate change is overwhelming. NASA data shows that global temperatures have risen by approximately 1.1 degrees Celsius since the late 19th century. Additionally, atmospheric CO2 levels have increased from 315 ppm in 1958 to over 420 ppm today. These changes correlate directly with increased industrial activity and fossil fuel consumption.\n\nCritics often point to natural climate variations as an explanation for current changes. However, climate scientists have thoroughly examined these factors and concluded that natural forces alone cannot account for the rapid warming observed since the mi

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [17]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [18]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [19]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<bos><start_of_turn>user\nPlease analyze the content quality of this essay:\n\nClimate change represents one of the most pressing challenges of our time. Human activities, particularly the burning of fossil fuels, have significantly increased greenhouse gas concentrations in the atmosphere. This has led to global warming, rising sea levels, and extreme weather events.\n\nThe evidence for human-caused climate change is overwhelming. NASA data shows that global temperatures have risen by approximately 1.1 degrees Celsius since the late 19th century. Additionally, atmospheric CO2 levels have increased from 315 ppm in 1958 to over 420 ppm today. These changes correlate directly with increased industrial activity and fossil fuel consumption.\n\nCritics often point to natural climate variations as an explanation for current changes. However, climate scientists have thoroughly examined these factors and concluded that natural forces alone cannot account for the rapid warming observed since t

Now let's print the masked out example - you should see only the answer is present:

In [20]:
tokenizer.decode([tokenizer.pad_token_id if x == -0 else x for x in trainer.train_dataset[0]["labels"]]).replace(tokenizer.pad_token, " ")

IndexError: Invalid key: 100 is out of bounds for size 2

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.283 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.237700
2,1.636400
3,1.766300
4,1.420700
5,1.235700
6,1.806600
7,1.010100
8,1.896600
9,1.464700
10,1.309700


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1068.4322 seconds used for training.
17.81 minutes used for training.
Peak reserved memory = 13.561 GB.
Peak reserved memory for training = 9.278 GB.
Peak reserved memory % of max memory = 91.995 %.
Peak reserved memory for training % of max memory = 62.94 %.


## Testing the Fine-tuned Content Analyzer

Now let's test our fine-tuned Gemma-3-4B model on the same essay to see how it performs compared to the original Gemini-based analyzer.

In [1]:
# Test the fine-tuned model on content analysis
from unsloth.chat_templates import get_chat_template

# Use the same sample essay we tested with the original analyzer
test_essay = """
Technology has fundamentally transformed education in the 21st century. From online learning platforms to interactive whiteboards, digital tools have become integral to modern classrooms. While these changes offer numerous benefits, they also present significant challenges that educators must address.

The advantages of educational technology are substantial. Students can access vast amounts of information instantly, engage with interactive content, and learn at their own pace through personalized platforms. Remote learning capabilities, highlighted during the COVID-19 pandemic, have made education more accessible to students regardless of geographical constraints.

However, the digital divide remains a critical concern. Not all students have equal access to technology or reliable internet connections, potentially exacerbating educational inequalities. Additionally, excessive screen time may impact students' attention spans and social development.

Effective integration of technology in education requires thoughtful planning and adequate teacher training. Rather than replacing traditional teaching methods entirely, technology should complement and enhance human instruction to create more engaging and effective learning experiences.
"""

# Create the prompt for content analysis
messages = [
    {"role": "user", "content": f"Please analyze the content quality of this essay:\n\n{test_essay}"}
]

# Apply the chat template
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate the content analysis
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and display the analysis
response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

print("Fine-tuned Gemma-3-4B Content Analysis:")
print("=" * 60)
print(response)
print("=" * 60)

ModuleNotFoundError: No module named 'unsloth'

## Model Performance Comparison

Let's compare the performance between:

1. **Original Gemini-2.5-Flash Content Analyzer**: Uses the full instruction prompt and Google's latest model
2. **Fine-tuned Gemma-3-4B**: Our customized model trained specifically on content analysis tasks

### Key Evaluation Criteria:

- **Accuracy**: How well does the analysis match expected content evaluation standards?
- **Consistency**: Does the model provide structured, consistent feedback format?
- **Depth**: Does the analysis cover all required aspects (argument quality, evidence, critical thinking, etc.)?
- **Specificity**: Are the recommendations actionable and specific?
- **Efficiency**: Speed and resource usage comparison

## Next Steps: Integrating with ADK Essay Analyzer

Once you're satisfied with the fine-tuned model's performance, you can integrate it into your ADK Essay Analyzer project:

### 1. Save and Export the Model

The fine-tuned model can be saved and used to replace or complement the existing Gemini-based content analyzer.

### 2. Model Integration Options

**Option A: Replace the existing model**
- Update the content analyzer to use the fine-tuned Gemma-3-4B
- Benefit: Lower costs, faster inference, customized behavior

**Option B: Hybrid approach**
- Use fine-tuned model for initial analysis
- Use Gemini for complex cases requiring broader knowledge
- Benefit: Best of both worlds

### 3. Performance Considerations

- **Speed**: Gemma-3-4B (especially quantized) is typically faster than API calls
- **Cost**: Local inference eliminates per-request API costs
- **Privacy**: All processing happens locally
- **Customization**: Model behavior specifically tailored to your needs

In [ ]:
# Example: Creating a new content analyzer using the fine-tuned model

class FineTunedContentAnalyzer:
    """Content analyzer using fine-tuned Gemma-3-4B model"""

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.name = "fine_tuned_content_analyzer"
        self.description = "Fine-tuned Gemma-3-4B for essay content analysis"

    def analyze(self, essay_text):
        """Analyze essay content using the fine-tuned model"""

        messages = [
            {
                "role": "user",
                "content": f"Please analyze the content quality of this essay:\n\n{essay_text}"
            }
        ]

        # Apply chat template
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        # Generate analysis
        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs,
                max_new_tokens=512,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        # Decode response
        response = self.tokenizer.decode(
            outputs[0][len(inputs[0]):],
            skip_special_tokens=True
        )

        return {"content_analysis": response}

# Create the fine-tuned analyzer instance
fine_tuned_analyzer = FineTunedContentAnalyzer(model, tokenizer)

print("Fine-tuned Content Analyzer created successfully!")
print(f"Name: {fine_tuned_analyzer.name}")
print(f"Description: {fine_tuned_analyzer.description}")

# Example usage
sample_result = fine_tuned_analyzer.analyze(test_essay[:200] + "...")
print(f"\nSample analysis preview: {sample_result['content_analysis'][:150]}...")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89...\n\nThis is the Fibonacci sequence, where each number is the sum of the two preceding ones.\n<end_of_turn>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down why the sky is blue! It's a fascinating phenomenon that boils down to a combination of physics and light. Here's the explanation:

**1. Sunlight and its Colors:**

* Sunlight, which appears white to us, is actually made up of *all* the


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3-ntu")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3/processor_config.json']

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "unsloth/Gemma-3-4B-bnb-4bit", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down what Gemma-3 is. It's a fascinating development in the world of AI, and here's a comprehensive overview:

**1. What it is:**

* **A Family of Open-Weight Language Models:** Gemma-3 isn't just *one* model


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )